In [84]:
!python -m spacy download en_core_web_md

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
pip install langdetect

In [86]:
data_root='/content/drive/My Drive/wiki'
data_file=open(data_root+'/intents-final.json').read()

In [87]:
import requests

url = "https://google-translation-unlimited.p.rapidapi.com/translate"

headers = {
	"content-type": "application/x-www-form-urlencoded",
	"X-RapidAPI-Key": RapidAPI-key,
	"X-RapidAPI-Host": "google-translation-unlimited.p.rapidapi.com"
}

In [88]:
from langdetect import detect

def translate_lang(text, lang):
    try:
        payload = {
          "texte": text,
          "to_lang": lang
          }
        translation = requests.post(url, data=payload, headers=headers).json()['translation_data']['translation']
        return translation
    except Exception as e:
        print("Error:", e)
        return None

In [89]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [90]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

def calculate_similarity(text1, text2):
    processed_text1 = preprocess_text(text1)
    processed_text2 = preprocess_text(text2)

    processed_texts = [processed_text1, processed_text2]

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_texts)

    similarity_matrix = cosine_similarity(tfidf_matrix)

    similarity_score = similarity_matrix[0][1]
    return similarity_score


In [92]:
import json
import spacy

intents = json.loads(data_file)
nlp = spacy.load('en_core_web_md')

def get_intent(user_input):
    max_similarity = 0
    intent_tag = None
    for intent in intents['intents']:
        for pattern in intent['patterns']:
          if user_input!=None and pattern!=None:
            similarity = calculate_similarity(user_input.lower(), pattern.lower())
            if similarity > max_similarity:
                max_similarity = similarity
                intent_tag = intent['tag']

    return intent_tag

def get_response(intent_tag):
    for intent in intents['intents']:
        if intent['tag'] == intent_tag:
            response = intent['responses'][0]
            return response

def chat():
    print("Hello! Welcome to the Wikipedia editing support chatbot.")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == 'exit':
            print("Goodbye! Don't hesitate to return if you need further assistance.")
            break

        detected_lang = detect(user_input)

        if detected_lang != 'en':
            translated_input = translate_lang(user_input, "en")
        else:
            translated_input = user_input

        intent_tag = get_intent(translated_input)
        response = get_response(intent_tag)

        if response==None:
          translated_response = "Sorry, I donot have an answer to that :("
        if detected_lang != 'en':
            translated_response = translate_lang(response, detected_lang)
        else:
            translated_response = response

        print("Bot:", translated_response)


if __name__ == "__main__":
    chat()


Hello! Welcome to the Wikipedia editing support chatbot.
You: hey, can you help me edit a page?
Bot: To edit the whole page, click the 'edit' tab at the top. To edit just one section, click the 'edit' link to the right of the section heading.
You: పేజీని ఎలా తొలగించాలి?
en
How to delete the page? heyy
te
పేజీ తొలగింపులపై సూచనల కోసం వికీపీడియా: తొలగింపు విధానాన్ని చూడండి.వ్యాసాలు సాధారణంగా వికీపీడియాలో నామినేట్ చేయబడతాయి: తొలగింపు కోసం వ్యాసాలు. heyy
Bot: పేజీ తొలగింపులపై సూచనల కోసం వికీపీడియా: తొలగింపు విధానాన్ని చూడండి.వ్యాసాలు సాధారణంగా వికీపీడియాలో నామినేట్ చేయబడతాయి: తొలగింపు కోసం వ్యాసాలు.
You: నేను వచనాన్ని బోల్డ్ లేదా ఇటాలిక్ ఎలా చేయాలి?
en
How do I do the text bold or italic? heyy
te
టూల్‌బార్‌లోని B మరియు I బటన్లను ఉపయోగించి వచనాన్ని బోల్డ్ లేదా ఇటాలిక్ చేయవచ్చు.ప్రత్యామ్నాయంగా, మీరు 'అపోస్ట్రోఫెస్' మధ్య వచనాన్ని ఉంచవచ్చు;ప్రతి వైపు మూడు బోల్డ్ కోసం మరియు ఇటాలిక్స్ కోసం ప్రతి వైపు రెండు. heyy
Bot: టూల్‌బార్‌లోని B మరియు I బటన్లను ఉపయోగించి వచనాన్ని బోల్డ్ లేదా ఇటాలిక్ చేయవచ్చు